<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/cross_validation%2C_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵.
#테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련세트를 또 나누는 것 > 검증세트
#훈련세트에서 모델을 훈련하고 검증세트로 모델을 평가, 매개변수를 사용해 훈련 세트와 검증세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련, 테스트세트에서 최종 점수를 평가> 실전과 유사함
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [5]:
# train_input과 train_target을 다시 train_test_split에 넣어 훈련 세트 sub_input, sub_target과 검증세트 val_input, val_target을 만든다.test_size=0.2를 하여 train)input의 20%를 val_input으로
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [6]:
#훈련세트와 검증 세트의 크기확인
print(sub_input.shape, val_input.shape) #훈련세트가 4157개로 줄고 검증 세트가 1040개가 됨

(4157, 3) (1040, 3)


In [7]:
#sub_input, sub_target, val_input, val_target을 사용해 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [8]:
#훈련세트를 k개의 부분으로 나눠서 교차 검증을 수행하는것을 k-폴드 교차 검증이라고 함.
#사이킷런에는 cross_validate()라는 교차 검증 함수가 있음
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) #평가할 모델 객체를 첫 번째 매개변수로 전달, 그 다음 앞에서처럼 직접검증세트를 뗴어 내지 않고 훈련세트 전체를 전달
print(scores)

{'fit_time': array([0.03036427, 0.01768899, 0.01130843, 0.0124464 , 0.01559067]), 'score_time': array([0.00187087, 0.00201273, 0.00247169, 0.0020256 , 0.00204206]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행
/cross_validate() 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환

In [9]:
import numpy as np #교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음. 이름은 test_score지만 검증 폴드의 점수임
print(np.mean(scores['test_score']))

0.855300214703487


cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. train_test_split()함수로 전체 데이터를 섞은 후 훈련 세트를 준비 했기 때문에 따로 섞을 필요가 없지만 만약 교차 검증을 할때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야함.
사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해줌,
#cross_validtate()함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용

In [18]:
#앞서 수행한 교차검증은 다음 코드와 동일
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [19]:
#만약 훈련세트를 섞은 후 10-폴드 교차검증을 수행하려면
splitter = StratifiedKFold(n_splits = 10, shuffle =True, random_state = 42)
scores = cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [24]:
#모델이 학습할 수 없어 사용자가 지정해야만 하는 파라미터=>하이퍼 파라미터
#이미 만들어진 도구를 사용하는 것이 편함 > 사이킷런에서 제공하는 그리드서치
from sklearn.model_selection import GridSearchCV
#먼저 GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듬
params = {'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]} #여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도

In [25]:
#GridSearchCV클래스에 탐색 대상 모델과 aprams 변수를 전달하여 그리드 서치 객체를 만듬
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
#GridSearchCV의 cv 매개변수 기본값은 5, 따라서 min_impurity)decrease값마다 5-폴드교차검증을 수행> 결국 5*5 = 25개의 모델을 훈련
#n_jobs는 병렬 실행에 사용할 CPU 코어의 수를 지정, 기본값은 1, -1로 지정하면 시스템에 있는 모든 코어를 사용

In [26]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [27]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
##사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련
#이 모델은 gs객체의 best_estimator_속성에 저장

0.9615162593804117


In [28]:
print(gs.best_params_) #그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장

{'min_impurity_decrease': 0.0001}


In [29]:
#각 매개변수에서 수행한 교차 검증의 평균점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어있음
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [31]:
#수동으로 고르는 것보다 넘파이argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출가능>인덱스를 사용해 params키에 저장된 매개변수를 출력가능
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [34]:
#결정트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001), #첫번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열
          'max_depth': range(5,20,1),
          'min_samples_split':range(2,100,10)}

In [35]:
#9*15*10 = 1350, 기본 5-폴드 교차검증을 수행하므로 1350*5 = 6750개의 모델이 만들어짐
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [37]:
print(gs.best_params_) #최상의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [38]:
#최상의 교차검증점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [39]:
#uniform은 실수값, randint는 정수값
from scipy.stats import uniform, randint

In [40]:
rgen = randint(0,10)
rgen.rvs(10)

array([1, 2, 2, 7, 7, 3, 8, 3, 3, 7])

In [41]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99, 115, 107, 116, 104,  85,  77, 124,  89,  84]))

In [42]:
ugen = uniform(0,10)
ugen.rvs(10)

array([7.46167346, 3.34004779, 1.88350944, 0.44300121, 4.64116707,
       8.03344533, 0.63477995, 2.6968752 , 8.75442231, 7.9297934 ])

In [43]:
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25),}

In [45]:
#params에 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 교차 검증을 수행, 최적의 매개변수 조합을 찾음
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter = 100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f485a89e980>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f48591037c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f4859103ca0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f4859103e80>},
                   random_state=42)

In [46]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [48]:
#최적의 교차검증점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [49]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
